# Train a Keras Model

There are some basic steps to train and test a machine learning model for any kind of problem.

1. Selection of machine learning model.


2. Train the model.


3. Test the model.


4. Evaluate the model.

This process is called the model train test evaluation flow. The testing data should be different than the training data as predicting correct results on unseen data shall truly determine the accuracy of the model.

Keras makes it simple to model a complex neural network and still gives you a large amount of control over the structure of each layer. Decide the structure of the neural network as follow:

- Number of input and output
- Number of hidden layers
- Number of nodes or activation units in each layer
- Type of the layers (Fully connected, convolutional, recurrent…etc.)
- Activation function for each layer (softmax, relu, leaky relu, sigmoid…etc.)

In [22]:
############################
## Preprocessing
############################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore')

# Load the data
data = pd.read_csv('ecommerce_customers')

# Check dimensions and data
print(data.head())
print(data.shape)

############################
## Split and scale 
############################

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Split training
# y = data['Yearly Amount Spent']
# X = data.drop(['Yearly Amount Spent'], axis=1)
y = data['Yearly Amount Spent'].values
X = data[['Avg. Session Length', 'Time on App', 'Time on Website','Length of Membership']].values

# Assign variables for test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

print('Shape of Y_training', y_train.shape)
print('Shape of Y_testing', y_test.shape)

# Reshape the data
y_train_re = np.reshape(y_train, (-1, 1)) # Data has to be 1D so use .values if didn't already
y_test_re = np.reshape(y_test, (-1, 1))

print(y_train_re.shape)
print(y_test_re.shape)

# Scale to a range from 0 to 1 for neural networks to work well
X_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
X_scaled_train = X_scaler.fit_transform(X_train)
y_scaled_train = y_scaler.fit_transform(y_train_re)

# Scale test data
X_scaled_test = X_scaler.fit_transform(X_test)
y_scaled_test = y_scaler.fit_transform(y_test_re)

print(X_scaled_test.shape)
print(y_scaled_test.shape)
print("Note: Y values were scaled by multiplying by {:.10f} and adding {:.4f}".format(y_scaler.scale_[0], y_scaler.min_[0]))

############################
## Model Development
############################

from keras.models import Sequential
from keras.layers import *

# Define the model
model = Sequential()
model.add(Dense(50, input_dim=4, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss="mean_squared_error", optimizer="adam")

# Train the model
model.fit(
    X_scaled_train,
    y_scaled_train,
    epochs=50,
    shuffle=True,
    verbose=2
)

# Evaluate the model
test_error_rate = model.evaluate(X_scaled_test, y_scaled_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

y_predicted_scaled = model.predict(X_scaled_test)

# Unscale the data back to it's original units (dollars)
y_predicted = y_scaler.inverse_transform(y_predicted_scaled)

# Grab just the first element of the first prediction 
prediction = y_predicted[0][0]
real_earnings = y_test[0]

print("Earnings Prediction for Proposed Product - ${}".format(prediction))
print("Real Value of the Product - ${}".format(real_earnings))

############################
## Save, Load, Predict
############################

# model.save("keras_model_1.h5")
# model = keras.models.load_model("keras_model_1.h5")
# predictions = model.predict(new_data)

                           Email  \
0      mstephenson@fernandez.com   
1              hduke@hotmail.com   
2               pallen@yahoo.com   
3        riverarebecca@gmail.com   
4  mstephens@davidson-herman.com   

                                             Address            Avatar  \
0     835 Frank Tunnel\r\nWrightmouth, MI 82180-9605            Violet   
1   4547 Archer Common\r\nDiazchester, CA 06566-8576         DarkGreen   
2  24645 Valerie Unions Suite 582\r\nCobbborough,...            Bisque   
3  1414 David Throughway\r\nPort Jason, OH 22070-...       SaddleBrown   
4  14023 Rodriguez Passage\r\nPort Jacobville, PR...  MediumAquaMarine   

   Avg. Session Length  Time on App  Time on Website  Length of Membership  \
0            34.497268    12.655651        39.577668              4.082621   
1            31.926272    11.109461        37.268959              2.664034   
2            33.000915    11.330278        37.110597              4.104543   
3            34.305557    

In [23]:
print("Earnings Prediction for Proposed Product - ${}".format(prediction))
print("Real Value of the Product - ${}".format(real_earnings))

Earnings Prediction for Proposed Product - $360.3992919921875
Real Value of the Product - $451.57568515949305
